Definition of extreme event according to Regulation EU 2017/1938 Article 6: 

"Extreme temperature duing a 7-day peak period with a statistical probability of one in 20 years"

We have 40 years of data ---> find the coldest 7-day peak period for each country and make it once in 40 years

In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
# quantile equivalent of 7-day period once in 20 years
q=7/365.25/20
print(q)

0.0009582477754962353


In [3]:
cwd=os.getcwd()
path_adjusted_temp=os.path.join(cwd, 'adjusted_ninja_temperature/')
all_csvs = glob.glob(path_adjusted_temp + "/*.csv")

countries=[filename[-6:-4] for filename in all_csvs]
print(countries)

['AT', 'BE', 'CZ', 'DE', 'DK', 'EE', 'GB', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LV', 'NL', 'PL', 'RO', 'SI', 'SK', 'CH', 'FI', 'NO', 'SE', 'PT', 'BG', 'ES', 'FR', 'LU', 'UA']


In [4]:
df_extreme={}

for country in countries:
    
    df_temp=pd.read_csv('adjusted_ninja_temperature/adjusted_ninja_temp_%s.csv' %country)
    df_temp['time']=pd.to_datetime(df_temp['time'])
    df_temp=df_temp.set_index('time')
    
    df_daily=df_temp.groupby(df_temp.index.date).mean()
    # take daily mean
    v_daily=df_daily.values
    # take 7 days average

    v_seven_days_average=np.array([
                v_daily[i:i+7].mean() for i in range(len(v_daily))]) # last 7 days are not 7-day averages but 
                                                                     # averages over the remaining days
    threshold=np.quantile(v_seven_days_average, q)        
    dates=df_daily.index[v_seven_days_average<threshold]
    averages=v_seven_days_average[v_seven_days_average<threshold]
    
    _df_extreme=pd.DataFrame(index=dates, columns=['7day_average'], data=averages)
    _df_extreme.index=pd.to_datetime(_df_extreme.index)
    _df_extreme.index.name='date'
    df_extreme[country]=_df_extreme
    
    print(country, threshold)
    
    for year, _df in _df_extreme.groupby(_df_extreme.index.year):
        for month, _df1 in _df.groupby(_df.index.month):
            print(_df1['7day_average'].idxmin(), _df1['7day_average'].min())

    

AT -10.71258214356232
1985-01-05 00:00:00 -13.190466068049124
1987-01-08 00:00:00 -10.999560913966919
1996-12-26 00:00:00 -11.477316803884063
2012-02-02 00:00:00 -10.96939939101035
BE -6.024112540659749
1985-01-13 00:00:00 -7.425347505988374
1987-01-11 00:00:00 -7.671913416443252
1996-12-31 00:00:00 -6.610377196416607
1997-01-01 00:00:00 -6.405845492933115
CZ -13.239544328915576
1985-01-05 00:00:00 -13.936553332308597
1987-01-10 00:00:00 -14.832213845247594
1996-12-26 00:00:00 -14.955217459475561
DE -9.097589066008569
1985-01-05 00:00:00 -9.879638564825148
1987-01-11 00:00:00 -10.926726755952147
1996-12-27 00:00:00 -9.536591638945946
DK -7.814014660162354
1982-01-05 00:00:00 -8.208026197661372
1985-01-04 00:00:00 -8.041313356744157
1985-02-08 00:00:00 -8.731105593219313
1987-01-09 00:00:00 -10.596961255698085
EE -17.17346294361783
1985-02-05 00:00:00 -18.132054356619044
1987-01-06 00:00:00 -22.7003336636667
2011-02-15 00:00:00 -17.52598739067706
2012-01-30 00:00:00 -17.178924860407392


In [5]:
# we have 40 years of data -> record the lowest two years to make it twice in 40 years 
df_peak_years=pd.DataFrame(index=countries, columns=['coldest_year', 'coldest_week_commencing',
                                                     'coldest_week_mean_temp', '2ndcoldest_year', 
                                                     '2ndcoldest_week_commencing', '2ndcoldest_week_mean_temp'])
for country in countries:
    
    _df=df_extreme[country]
    
    coldest_average=_df['7day_average'].min()
    coldest_day=_df['7day_average'].idxmin()
        
    
    _df=_df.loc[(_df.index<coldest_day-pd.Timedelta('30days')) | (_df.index>coldest_day+pd.Timedelta('30days'))]
    
    coldest2_average=_df['7day_average'].min()
    coldest2_day=_df['7day_average'].idxmin()
    
    df_peak_years.loc[country]=[coldest_day.year, coldest_day, coldest_average,
                               coldest2_day.year, coldest2_day, coldest2_average]
df_peak_years

,coldest_year,coldest_week_commencing,coldest_week_mean_temp,2ndcoldest_year,2ndcoldest_week_commencing,2ndcoldest_week_mean_temp
AT,1985,1985-01-05 00:00:00,-13.190466,1996,1996-12-26 00:00:00,-11.477317
BE,1987,1987-01-11 00:00:00,-7.671913,1985,1985-01-13 00:00:00,-7.425348
CZ,1996,1996-12-26 00:00:00,-14.955217,1987,1987-01-10 00:00:00,-14.832214
DE,1987,1987-01-11 00:00:00,-10.926727,1985,1985-01-05 00:00:00,-9.879639
DK,1987,1987-01-09 00:00:00,-10.596961,1985,1985-02-08 00:00:00,-8.731106
EE,1987,1987-01-06 00:00:00,-22.700334,1985,1985-02-05 00:00:00,-18.132054
GB,1982,1982-01-08 00:00:00,-3.036316,1987,1987-01-11 00:00:00,-2.882142
GR,1987,1987-03-04 00:00:00,-0.015684,2002,2002-01-02 00:00:00,0.923749
HR,1985,1985-01-06 00:00:00,-9.300267,2012,2012-02-04 00:00:00,-8.205949
HU,1996,1996-12-25 00:00:00,-12.337935,1985,1985-01-06 00:00:00,-10.621559


In [6]:
# save 
df_peak_years.to_csv('peak_years_80to19.csv')